In [2]:
!nvidia-smi

Fri Mar  7 16:17:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77.01              Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P8              8W /   75W |     359MiB /   4096MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [32]:
# Custom autograd functions for binary operations
class SignSTE(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.sign(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clone()

class BinaryWeightSTE(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.sign(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clone()

def Binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
        return tensor.sign()
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)
# Binary convolutional layer
class BinarizeLinear(nn.Linear):

    def __init__(self, *kargs, **kwargs):
        super(BinarizeLinear, self).__init__(*kargs, **kwargs)

    def forward(self, input):

        if input.size(1) != 784:
            input.data=Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.org)
        out = F.linear(input, self.weight)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)

        return out

class BinaryConv2d(nn.Conv2d):

    def __init__(self, *kargs, **kwargs):
        super(BinaryConv2d, self).__init__(*kargs, **kwargs)

    def forward(self, input):
        if input.size(1) != 3:
            input.data = Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.org)

        out = F.conv2d(input, self.weight, None, self.stride, self.padding, self.dilation, self.groups)

        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1, 1, 1).expand_as(out)
        return out

# Binary CNN architecture
class BinaryCNN(nn.Module):
    def __init__(self):
        super(BinaryCNN, self).__init__()
        self.ratioInfl=0.0625
        self.features = nn.Sequential(
            BinaryConv2d(1, int(64*self.ratioInfl), kernel_size=3, stride=1, padding=1),

            nn.BatchNorm2d(int(64*self.ratioInfl)),
            nn.ReLU(inplace=True),
            BinaryConv2d(int(64*self.ratioInfl), int(64*self.ratioInfl), kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(int(64*self.ratioInfl)),
            nn.ReLU(inplace=True),

            BinaryConv2d(int(64*self.ratioInfl), int(128*self.ratioInfl), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(128*self.ratioInfl)),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            BinaryConv2d(int(128*self.ratioInfl), int(128*self.ratioInfl), kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(int(128*self.ratioInfl)),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            BinaryConv2d(int(128*self.ratioInfl), 16, kernel_size=3, padding=1),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.classifier = nn.Sequential(
            BinarizeLinear(16 , 10),
            #nn.ReLU(inplace=True),
            #nn.Dropout(0.5),
            #BinarizeLinear(16, num_classes),
            #nn.BatchNorm1d(10),
            nn.LogSoftmax()
        )

    def forward(self, x):
        x = self.features(x)
        #x = x.view(-1, 256*3*3)
        x = torch.squeeze(x)
        x = self.classifier(x)
        return x

In [75]:
# Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)


In [76]:
print(len(train_dataset))
print(len(test_dataset))
print(test_dataset[0][0])

60000
10000
tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000,

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
mnist_og = test_dataset[0][0]
# plt.imshow(mnist_og, cmap=cm.Greys_r)
# plt.show()
plt.imshow(mnist_og, interpolation='none')
plt.show()

In [40]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BinaryCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [67]:
# Training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Testing loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
# Run training and testing
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.198937
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.187845
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.267798
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.290514
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.231546

Test set: Average loss: 2.3462, Accuracy: 1183/10000 (11.83%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.214943
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.239225
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.259748
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.169056
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.228522

Test set: Average loss: 2.2099, Accuracy: 1650/10000 (16.50%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.193044
Train Epoch: 3 [12800/60000 (21%)]	Loss: 2.206994
Train Epoch: 3 [25600/60000 (43%)]	Loss: 2.323282
Train Epoch: 3 [38400/60000 (64%)]	Loss: 2.398580
Train Epoch: 3 [51200/60000 (85%)]	Loss: 2.282883

Test set: Average loss: 2.6631, Accuracy: 763/10000 (7.63%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.198756
Train Epoch: 4 [12800/60000 (2

In [78]:
global best_prec1
best_prec1 = 0

import os
import shutil
import time
from torch.autograd import Variable
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.float().topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.contiguous().view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res
    
def save_checkpoint(state, is_best, path='.', filename='checkpoint.pth.tar', save_all=False):
    filename = os.path.join(path, filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(path, 'model_best.pth.tar'))
    if save_all:
        shutil.copyfile(filename, os.path.join(
            path, 'checkpoint_epoch_%s.pth.tar' % state['epoch']))
        
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def forward(data_loader, model, criterion, epoch=0, training=True, optimizer=None):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end = time.time()
    for i, (inputs, target) in enumerate(data_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        target = target.cuda()

        if not training:
            with torch.no_grad():
                input_var = Variable(inputs.type(torch.cuda.FloatTensor))
                target_var = Variable(target)
                # compute output
                output = model(input_var)
        else:
                input_var = Variable(inputs.type(torch.cuda.FloatTensor))
                target_var = Variable(target)
                # compute output
                output = model(input_var)


        loss = criterion(output, target_var)
        if type(output) is list:
            output = output[0]

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        if training:
            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            for p in list(model.parameters()):
                if hasattr(p,'org'):
                    p.data.copy_(p.org)
            optimizer.step()
            for p in list(model.parameters()):
                if hasattr(p,'org'):
                    p.org.copy_(p.data.clamp_(-1,1))


        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('{phase} - Epoch: [{0}][{1}/{2}]\t'
                         'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                         'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                         'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                         'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                         'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                             epoch, i, len(data_loader),
                             phase='TRAINING' if training else 'EVALUATING',
                             batch_time=batch_time,
                             data_time=data_time, loss=losses, top1=top1, top5=top5))

    return losses.avg, top1.avg, top5.avg
    
def train(data_loader, model, criterion, epoch, optimizer):
    # switch to train mode
    model.train()
    return forward(data_loader, model, criterion, epoch, training=True, optimizer=optimizer)


def validate(data_loader, model, criterion, epoch):
    # switch to evaluate mode
    model.eval()
    return forward(data_loader, model, criterion, epoch, training=False, optimizer=None)
    
for epoch in range(1, 100):

        # train for one epoch
        train_loss, train_prec1, train_prec5 = train(train_loader, model, criterion, epoch, optimizer)

        # evaluate on validation set
        val_loss, val_prec1, val_prec5 = validate(test_loader, model, criterion, epoch)

        # remember best prec@1 and save checkpoint
        is_best = val_prec1 > best_prec1
        best_prec1 = max(val_prec1, best_prec1)

        save_checkpoint({
            'epoch': epoch + 1,
            'model': 'BCNN',
            'config': 'None',
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, path='weight/')
        print('\n Epoch: {0}\t'
                     'Training Loss {train_loss:.4f} \t'
                     'Training Prec@1 {train_prec1:.3f} \t'
                     'Training Prec@5 {train_prec5:.3f} \t'
                     'Validation Loss {val_loss:.4f} \t'
                     'Validation Prec@1 {val_prec1:.3f} \t'
                     'Validation Prec@5 {val_prec5:.3f} \n'
                     .format(epoch + 1, train_loss=train_loss, val_loss=val_loss,
                             train_prec1=train_prec1, val_prec1=val_prec1,
                             train_prec5=train_prec5, val_prec5=val_prec5))




TRAINING - Epoch: [1][0/469]	Time 0.227 (0.227)	Data 0.201 (0.201)	Loss 2.2093 (2.2093)	Prec@1 21.875 (21.875)	Prec@5 57.812 (57.812)
TRAINING - Epoch: [1][100/469]	Time 0.007 (0.013)	Data 0.000 (0.004)	Loss 2.2623 (2.2522)	Prec@1 14.062 (15.153)	Prec@5 59.375 (58.617)
TRAINING - Epoch: [1][200/469]	Time 0.008 (0.011)	Data 0.001 (0.003)	Loss 2.2777 (2.2507)	Prec@1 14.062 (15.520)	Prec@5 60.156 (58.244)
TRAINING - Epoch: [1][300/469]	Time 0.019 (0.011)	Data 0.005 (0.003)	Loss 2.2412 (2.2477)	Prec@1 21.094 (15.812)	Prec@5 62.500 (58.493)
TRAINING - Epoch: [1][400/469]	Time 0.017 (0.011)	Data 0.007 (0.003)	Loss 2.2119 (2.2478)	Prec@1 14.844 (15.775)	Prec@5 60.156 (58.598)
EVALUATING - Epoch: [1][0/79]	Time 0.059 (0.059)	Data 0.054 (0.054)	Loss 2.1893 (2.1893)	Prec@1 19.531 (19.531)	Prec@5 53.125 (53.125)

 Epoch: 2	Training Loss 2.2453 	Training Prec@1 15.852 	Training Prec@5 58.802 	Validation Loss 2.2059 	Validation Prec@1 18.490 	Validation Prec@5 58.130 

TRAINING - Epoch: [2][0/469]	

In [ ]:

mnist_og = test_dataset[0]
plt.imshow(mnist_og, cmap=cm.Greys_r)
plt.show()

In [79]:
test()


Test set: Average loss: 2.8005, Accuracy: 809/10000 (8.09%)



In [82]:
import numpy as np
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[[[-1., -1., -1.],
          [-1.,  1.,  1.],
          [ 1.,  1., -1.]]],


        [[[-1., -1.,  1.],
          [-1.,  1.,  1.],
          [ 1.,  1., -1.]]],


        [[[-1., -1., -1.],
          [-1.,  1.,  1.],
          [ 1.,  1., -1.]]],


        [[[-1., -1., -1.],
          [ 1.,  1.,  1.],
          [ 1.,  1., -1.]]]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.3150,  0.8517, -0.2110,  0.0531], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([38.4891, 43.4622, 41.7565, 39.5340], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([-5.9068, 16.8850, -3.2084, -8.7954], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([[[[-1.,  1.,  1.],
          [ 1., -1., -1.],
          [ 1., -1., -1.]],

         [[-1., -1.,  1.],
          [-1.,  1., -1.],
          [ 1., -1., -1.]],

         [[-1.,  1.,  1.],
          [ 1.,  1.,  1.],
          [ 1., -1., -1.]],